In [1]:
import numpy as np
import librosa
import os
import glob
import pandas as pd
from tqdm import tqdm

In [2]:
!pip install librosa

    100% |████████████████████████████████| 1.6MB 21.2MB/s ta 0:00:01
    100% |████████████████████████████████| 286kB 37.9MB/s ta 0:00:01
    100% |████████████████████████████████| 327kB 38.8MB/s ta 0:00:01
  Running setup.py bdist_wheel for librosa ... done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/18/b8/10/f0f8f6ac60668a5cd75596cf14c25bb6b3ea1ecd815f058b7e
  Running setup.py bdist_wheel for audioread ... done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/53/02/90/7b5c4081b7470c550ab605f600bad237dde12a6b8999b11f50
  Running setup.py bdist_wheel for resampy ... done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
Successfully built librosa audioread resampy
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [38]:
audio = '/home/ubuntu/UrbanSound8K/audio'
folders = ["fold1","fold2","fold3","fold4","fold5","fold6","fold7","fold8","fold9","fold10"]

In [36]:
def get_audio_features(parent_dir, sub_dirs, file_ext="*.wav"):
    features = np.empty((0,88))
    labels = []
    fold = []
    for label, sub_dir in enumerate(sub_dirs):
        print(sub_dir)
        for file in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            librosa_load, librosa_sampling_rate = librosa.load(file, mono=True)
            stft = np.abs(librosa.stft(librosa_load))
            mfcc_mean = np.mean(librosa.feature.mfcc(y=librosa_load, sr=librosa_sampling_rate, n_mfcc=13).T,axis=0)
            mfcc_std = np.std(librosa.feature.mfcc(y=librosa_load, sr=librosa_sampling_rate, n_mfcc=13).T,axis=0)
            contrast_mean = np.mean(librosa.feature.spectral_contrast(S=stft, sr=librosa_sampling_rate).T,axis=0)
            contrast_std = np.std(librosa.feature.spectral_contrast(S=stft, sr=librosa_sampling_rate).T,axis=0)
            chroma_mean = np.mean(librosa.feature.chroma_stft(S=stft, sr=librosa_sampling_rate).T,axis=0)
            chroma_std = np.std(librosa.feature.chroma_stft(S=stft, sr=librosa_sampling_rate).T,axis=0)
            rolloff_mean = np.mean(librosa.feature.chroma_stft(S=stft, sr=librosa_sampling_rate).T,axis=0)
            rolloff_std = np.std(librosa.feature.chroma_stft(S=stft, sr=librosa_sampling_rate).T,axis=0)
            feature_cols = np.hstack([mfcc_mean, mfcc_std, contrast_mean, contrast_std, 
                                      chroma_mean, chroma_std, rolloff_mean, rolloff_std])
            features = np.vstack((features, feature_cols))
            y = (file.split('/')[6].split('-')[1])
            fold.append(sub_dir)
            labels.append(int(y))
    return feature_cols, labels, fold

In [37]:
features, labels, folds = get_audio_features(audio, folders)

fold1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45


KeyboardInterrupt: 

In [23]:
labels

[6,
 2,
 4,
 2,
 7,
 8,
 9,
 5,
 5,
 9,
 0,
 2,
 2,
 8,
 2,
 5,
 2,
 7,
 2,
 4,
 2,
 7,
 9,
 9,
 9,
 1,
 7,
 0,
 0,
 2,
 7,
 0,
 9,
 7,
 6,
 9,
 4,
 2,
 1,
 5,
 7,
 5,
 2,
 2,
 0,
 2,
 5,
 9,
 4,
 2,
 2,
 1,
 0,
 0,
 7,
 0,
 2,
 4,
 4,
 3,
 3,
 0,
 9,
 9,
 0,
 2,
 1,
 4,
 9,
 0,
 7,
 1,
 4,
 5,
 4,
 9,
 4,
 7,
 8,
 7,
 1,
 5,
 4,
 2,
 3,
 8,
 9,
 6,
 9,
 9,
 3,
 8,
 4,
 8,
 2,
 0,
 7,
 3,
 7,
 6,
 7,
 4,
 5,
 8,
 0,
 9,
 4,
 2,
 6,
 7,
 7,
 8,
 9,
 4,
 0,
 1,
 5,
 4,
 9,
 2,
 3,
 5,
 7,
 8,
 3,
 8,
 3,
 1,
 8,
 3,
 3,
 8,
 7,
 4,
 8,
 2,
 5,
 8,
 2,
 0,
 1,
 2,
 8,
 5,
 6,
 4,
 4,
 4,
 9,
 3,
 3,
 3,
 9,
 7,
 0,
 1,
 3,
 9,
 0,
 2,
 5,
 2,
 5,
 9,
 3,
 2,
 3,
 4,
 0,
 5,
 4,
 9,
 5,
 7,
 4,
 3,
 9,
 0,
 0,
 9,
 9,
 7,
 4,
 3,
 5,
 6,
 4,
 2,
 3,
 2,
 7,
 8,
 2,
 2,
 9,
 7,
 7,
 6,
 7,
 6,
 9,
 9,
 2,
 4,
 7,
 3,
 3,
 9,
 4,
 9,
 0,
 5,
 3,
 2,
 5,
 4,
 0,
 7,
 7,
 0,
 2,
 6,
 3,
 7,
 8,
 9,
 8,
 9,
 2,
 9,
 7,
 0,
 1,
 5,
 4,
 9,
 1,
 3,
 8,
 2,
 1,
 3,
 0,
 0,
 0,
 1,
 1,
 6,
 5,
 5,


In [78]:
# np.savetxt("features.csv", np_out, delimiter=",")

In [3]:
df = pd.read_csv('./features.csv')

In [4]:
df.head()

,-3.474220670334815964e+00,2.857897970375768981e+00,-3.590239939233062838e+00,1.746021854093317449e+00,-9.948786878218937035e-01,6.316859517701494120e-01,-6.917352256234181596e-01,3.332999351777252706e-01,-2.457553010090723633e-01,5.544724016360862845e-02,...,-6.867144938376570362e-01,-4.853452426389289753e-01,7.340359895996497741e-01,-2.705942364236723630e-01,-5.880346485479942542e-01,-5.869473092834435191e-01,-6.011493100330262962e-01,-4.840346262677158173e-01,4.173106589610258155e-01,2.113449471594827322e+00
0,-5.373538,2.919077,-0.661504,0.601540,0.045294,0.148810,0.424438,0.408690,0.343655,0.316244,...,1.535709,0.588437,-0.513087,-1.036728,-0.957482,-0.267436,0.025869,-0.010860,0.282955,1.963683
1,-2.552239,4.187415,-3.076004,0.028714,-0.860569,-0.170561,-0.745565,-0.078530,-0.903930,-0.077292,...,-1.310567,0.259663,1.451087,-0.747415,-0.585809,-0.460454,-0.469024,-0.167301,0.296919,2.133084
2,-6.058441,1.115634,0.220812,0.633115,0.193441,-0.011261,0.013524,-0.009777,0.005731,-0.005094,...,-1.345980,-0.263892,0.123302,-0.319602,-0.743017,-0.097980,-0.570727,-0.337132,-0.139980,2.208438
3,-3.557316,4.863821,-1.032547,0.378554,0.261618,0.140604,0.189797,0.177898,-0.032236,0.470381,...,1.507666,1.346528,0.336574,0.113144,-1.086707,-0.487760,-0.447089,-0.178485,0.004448,2.082449
4,-5.919869,1.461585,-0.569476,-0.101312,-0.131756,0.502519,0.554460,0.487312,0.079839,0.261224,...,-0.942560,-0.866023,-0.433400,-0.581125,-1.364023,-0.947368,0.517501,0.243292,0.675280,1.456443


In [8]:
col_list = []
for i in range(1, 41):
    col_list.append('mfcc' + str(i))
for i in range(1, 13):
    col_list.append('chroma' + str(i))
for i in range(1, 8):
    col_list.append('contrast' + str(i))

In [10]:
len(col_list)

59

In [87]:
sr = pd.Series(labels)

In [88]:
df['label'] = sr

In [58]:
# df = df.drop(labels=0, axis=1)

In [89]:
df.head()

,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,...,chroma11,chroma12,contrast1,contrast2,contrast3,contrast4,contrast5,contrast6,contrast7,label
0,-5.373538,2.919077,-0.661504,0.601540,0.045294,0.148810,0.424438,0.408690,0.343655,0.316244,...,0.588437,-0.513087,-1.036728,-0.957482,-0.267436,0.025869,-0.010860,0.282955,1.963683,6
1,-2.552239,4.187415,-3.076004,0.028714,-0.860569,-0.170561,-0.745565,-0.078530,-0.903930,-0.077292,...,0.259663,1.451087,-0.747415,-0.585809,-0.460454,-0.469024,-0.167301,0.296919,2.133084,2
2,-6.058441,1.115634,0.220812,0.633115,0.193441,-0.011261,0.013524,-0.009777,0.005731,-0.005094,...,-0.263892,0.123302,-0.319602,-0.743017,-0.097980,-0.570727,-0.337132,-0.139980,2.208438,4
3,-3.557316,4.863821,-1.032547,0.378554,0.261618,0.140604,0.189797,0.177898,-0.032236,0.470381,...,1.346528,0.336574,0.113144,-1.086707,-0.487760,-0.447089,-0.178485,0.004448,2.082449,2
4,-5.919869,1.461585,-0.569476,-0.101312,-0.131756,0.502519,0.554460,0.487312,0.079839,0.261224,...,-0.866023,-0.433400,-0.581125,-1.364023,-0.947368,0.517501,0.243292,0.675280,1.456443,7
